In [9]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [10]:
# poetry add pydantic
# %pip install pydantic 

from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema import OutputParserException

from pydantic import BaseModel, Field
from typing import List
import json


In [11]:
# 출력 구조를 정의하는 Pydantic 모델
class StudentInfos(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이")
    major: str  = Field(description="전공")
    hobbies: List[str] = Field(description="취미")
    goal: str = Field(description="목표")
    
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=StudentInfos)

# 프롬프트 템플릿 설정
template = """
다음 학생의 자기소개에서 정보를 추출하여 구조화 해주세요.

자기소개: {introduction}

{format_instructions}

중요사항:
- 이름, 나이, 전공, 취미, 목표를 정확히 추출하세요
- 취미는 리스트 형태로 여러 개 추출하세요
- 명시되지 않은 정보는 "정보 없음"으로 표시하세요
"""

prompt = ChatPromptTemplate.from_template(template)
# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(format_instructions=parser.get_format_instructions())



In [12]:
# ChatOpenAI 모델 초기화
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 구성 및 실행
chain = prompt | model | parser


def extract_student_info(introduction_text):
    """학생 자기소개에서 정보를 추출하는 함수"""
    try:
        result = chain.invoke({"introduction": introduction_text})
        return result, None
    except Exception as e:
        return None, f"오류: {str(e)}"
    
test_introductions = [
    "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다.",
    
    "제 이름은 박지연이고, 올해 20세입니다. 경영학과에 다니고 있고, 독서와 여행, 요리를 즐깁니다. 미래에는 글로벌 기업의 CEO가 되고 싶어요.",
    
    "이수진입니다. 나이는 23살이고 미술을 전공하고 있습니다. 그림 그리기, 사진 찍기, 전시회 관람이 취미예요. 유명한 작가가 되어 많은 사람들에게 감동을 주고 싶습니다.",
    
    "안녕하세요~ 저는 최진우예요. 21살이고 기계공학과 학생입니다. 축구, 농구 같은 운동을 좋아하고 음악 듣기도 즐겨해요. 앞으로 로봇 엔지니어가 되는 게 꿈이에요."
]

print("=== 학생 정보 구조화 시스템 ===\n")

for i, introduction in enumerate(test_introductions, 1):
    print(f" 테스트 케이스 {i}")
    print(f"자기소개: {introduction}")
    print()
    
    # 정보 추출
    student_info, error = extract_student_info(introduction)
    
    if student_info:
        print(" 추출된 정보:")
        print(f" 이름: {student_info.name}")
        print(f" 나이: {student_info.age}세")
        print(f" 전공: {student_info.major}")
        print(f" 취미: {', '.join(student_info.hobbies)}")
        print(f" 목표: {student_info.goal}")
        
        # JSON 형태로도 출력
        print("\n JSON 형태:")
        student_dict = student_info.dict()
        print(json.dumps(student_dict, ensure_ascii=False, indent=2))
        
    else:
        print(f" 추출 실패: {error}")
    
    print("="*80)
    print()

=== 학생 정보 구조화 시스템 ===

 테스트 케이스 1
자기소개: 안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다.

 추출된 정보:
 이름: 김민수
 나이: 22세
 전공: 컴퓨터공학
 취미: 게임하기, 영화보기, 코딩
 목표: 훌륭한 개발자가 되는 것

 JSON 형태:
{
  "name": "김민수",
  "age": 22,
  "major": "컴퓨터공학",
  "hobbies": [
    "게임하기",
    "영화보기",
    "코딩"
  ],
  "goal": "훌륭한 개발자가 되는 것"
}

 테스트 케이스 2
자기소개: 제 이름은 박지연이고, 올해 20세입니다. 경영학과에 다니고 있고, 독서와 여행, 요리를 즐깁니다. 미래에는 글로벌 기업의 CEO가 되고 싶어요.



C:\Users\user\AppData\Local\Temp\ipykernel_5476\3585669547.py:51: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  student_dict = student_info.dict()


 추출된 정보:
 이름: 박지연
 나이: 20세
 전공: 경영학과
 취미: 독서, 여행, 요리
 목표: 글로벌 기업의 CEO가 되고 싶어요

 JSON 형태:
{
  "name": "박지연",
  "age": 20,
  "major": "경영학과",
  "hobbies": [
    "독서",
    "여행",
    "요리"
  ],
  "goal": "글로벌 기업의 CEO가 되고 싶어요"
}

 테스트 케이스 3
자기소개: 이수진입니다. 나이는 23살이고 미술을 전공하고 있습니다. 그림 그리기, 사진 찍기, 전시회 관람이 취미예요. 유명한 작가가 되어 많은 사람들에게 감동을 주고 싶습니다.



C:\Users\user\AppData\Local\Temp\ipykernel_5476\3585669547.py:51: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  student_dict = student_info.dict()


 추출된 정보:
 이름: 이수진
 나이: 23세
 전공: 미술
 취미: 그림 그리기, 사진 찍기, 전시회 관람
 목표: 유명한 작가가 되어 많은 사람들에게 감동을 주고 싶습니다.

 JSON 형태:
{
  "name": "이수진",
  "age": 23,
  "major": "미술",
  "hobbies": [
    "그림 그리기",
    "사진 찍기",
    "전시회 관람"
  ],
  "goal": "유명한 작가가 되어 많은 사람들에게 감동을 주고 싶습니다."
}

 테스트 케이스 4
자기소개: 안녕하세요~ 저는 최진우예요. 21살이고 기계공학과 학생입니다. 축구, 농구 같은 운동을 좋아하고 음악 듣기도 즐겨해요. 앞으로 로봇 엔지니어가 되는 게 꿈이에요.



C:\Users\user\AppData\Local\Temp\ipykernel_5476\3585669547.py:51: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  student_dict = student_info.dict()


 추출된 정보:
 이름: 최진우
 나이: 21세
 전공: 기계공학과
 취미: 축구, 농구, 음악 듣기
 목표: 로봇 엔지니어

 JSON 형태:
{
  "name": "최진우",
  "age": 21,
  "major": "기계공학과",
  "hobbies": [
    "축구",
    "농구",
    "음악 듣기"
  ],
  "goal": "로봇 엔지니어"
}



C:\Users\user\AppData\Local\Temp\ipykernel_5476\3585669547.py:51: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  student_dict = student_info.dict()
